# Control of gopigo3 in RaspberryPi

Autonomous navigation refers that the robot is able to move autonomously around the environment avoiding any obstacle.

In a hospital, a delivery robot carries samples or food from one room to another. 

The main objectives are:
- Create a real robot (gopigo3))
- Install the onboard raspberrypi with ROS Melodic, gopigo3 drivers and sensors
- perform autonomous navigation in the Hospital environment
- generate and store a map of the hospital 
- use SLAM (Simultaneous Localization and Mapping) techniques to find an optimal trajectory to reach a speciffic hospital target position

let's see how to fulfill these objectives

![Getting Started](./Images/1_gopigo3_UB.png)

### RBPi install

You can obtain an image:

https://ubarcelona-my.sharepoint.com/personal/albertalvarez_ub_edu/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Falbertalvarez%5Fub%5Fedu%2FDocuments%2FPublic%2FRPi%20images&originalPath=aHR0cHM6Ly91YmFyY2Vsb25hLW15LnNoYXJlcG9pbnQuY29tLzpmOi9nL3BlcnNvbmFsL2FsYmVydGFsdmFyZXpfdWJfZWR1L0VxcE1HazB6WnhaTGtKYmZhTWxuZFZ3QjAxNFBpaGRhVVZCWG4yVGItTnN5Z0E_cnRpbWU9UDMtekFLcWwyRWc

or in this address:

https://ubarcelona-my.sharepoint.com/:f:/r/personal/albertalvarez_ub_edu/Documents/Public/RPi%20images?csf=1&web=1&e=39e4gc

If you want to install all the packages step by step. Refer to the document: https://github.com/Albert-Alvarez/rUBot



The RBPi is connected to a local network generated by a router: 
- with SSID name: roboticsUB 
- with the password "CorrePiCorre"
- and the IP: 192.168.4.1

To connect to the raspberrypi, you will need to open a terminal. We suggest to use:
- Remote desktop VNC. You will have access to folder tree structure and terminal
- "Terminal" application where you configure a white board terminal. A ssh connection will be needed.

In order to see the folder tree structure and terminals if you do not use the VNC, we propose:
- Standard "Fitxers" application and terminator with black board configuration
- New "Fitxers" application and "Terminal" with white board configuration


![Getting Started](./Images/2_fitxers.png)

If you need a internet connection, you will need to use the internet cable. To prepare the RaspberryPi workspace you have 2 options:

- with internet cable you can prepare the raspberrypi workspace directly in the RBPi
- you prepare the raspberrypi workspace in the PC and copy it to RBPi.

![Getting Started](./Images/2_ssh2pi.png)

### ROS workspace in Raspberrypi
First, let's prepare your raspberrypi with the required ROS workspace and packages needed for the navigation:


In [ ]:
mkdir pi_ws
cd pi_ws
mkdir src
catkin_make

copy in ~.bashrc the lines:



In [ ]:
source ~/pi_ws/devel/setup.sh
export ROS_IP=192.168.4.1
export ROS_MASTER_URI=http://192.168.4.1:11311


Be sure that in the PC the .bashrc knows that the roscore runs in the RaspberryPi:

In [ ]:
source ~/rubot_gopigo_ws/devel/setup.sh
export ROS_IP=192.168.18.84
export ROS_MASTER_URI=http://192.168.4.1:11311

## Install needed packages

To perform autonomous Navigation, different packages are needed:
- change to directory ~/pi_ws/src
- prepare the raspberrypi workspace in the PC and copy it to RBPi
- compile with catkin_make

You can install the diferent packages following the instructions:

### Install gopigo3 node

In [ ]:
git clone https://github.com/ros-gopigo/gopigo3_node
git clone --single-branch --branch=kinetic-devel https://github.com/ros-teleop/teleop_tools.git

### Install Package raspicam

In [ ]:
git clone https://github.com/UbiquityRobotics/raspicam_node.git
git clone --single-branch --branch=indigo-devel https://github.com/ros-perception/image_transport_plugins.git
git clone --single-branch --branch=melodic https://github.com/ros-perception/vision_opencv.git
git clone --single-branch --branch=hydro-devel https://github.com/ros-perception/image_common.git

### Install Package LIDAR

In [ ]:
git clone https://github.com/EAIBOT/ydlidar.git

Plug the LIDAR USB and we want pi and dialout as owner/owner group

In [ ]:
ls -la /dev/ttyUSB*
sudo chown pi:dialout /dev/ttyUSB0
roscd ydlidar/startup
sudo chmod a+rwx ./*
sudo sh initenv.sh

Ready to execute the LIDAR node

In [ ]:
roslaunch ydlidar lidar.launch
rosrun ydlidar ydlidar_client

## Setup gopigo3 with sensors

We will execute in different terminals:
- the gopigo3 node for driving control
- the ydlidar node (or rplidar)
- the raspicam node

In [ ]:
roslaunch gopigo3_node gopigo3.launch
roslaunch ydlidar lidar.launch (or roslaunch rplidar_ros rplidar.launch)
roslaunch raspicam_node camerav2_1280x960_10fps.launch enable_raw:=true camera_frame_id:="laser_frame"

we can see now the nodes that are running in our ROS workspace:

![Getting Started](./Images/2_nodes_cam.png)

A launch file is created to integrate all the needed roslaunch commands:

In [ ]:
roslaunch obstacle_avoidance obstacle_avoidance_node.launch distance_laser:=0.2 speed_factor:=1.3

In [ ]:
<launch>
  <node name="gopigo3" pkg="gopigo3_node" type="gopigo3_driver.py" />
  <include file="$(find rpydlidar_ros)/launch/rplidar.launch" />
    <arg name="distance_laser" default="0.3" />
    <arg name="speed_factor" default="0.2" />
    <arg name="forward_speed" default="3.8" />
    <arg name="backward_speed" default="-1.5" />
    <arg name="rotation_speed" default="20" />
    <node name="obstacle_avoidance" pkg="obstacle_avoidance" type="obstacle_avoidance_node.py" output="screen" >
        <param name="distance_laser" value="$(arg distance_laser)"/>
        <param name="speed_factor" value="$(arg speed_factor)"/>
        <param name="forward_speed" value="$(arg forward_speed)"/>
        <param name="backward_speed" value="$(arg backward_speed)"/>
        <param name="rotation_speed" value="$(arg rotation_speed)"/>
    </node>    
</launch>

In order to see the rubot with the topics information we will use rviz.

In rviz, select the fixed frame to "laser_frame", and add Camera and LaserScan with the corresponding topics names.

![Getting Started](./Images/2_pi_rviz.png)

### Laboratory Project

There are 2 main tasks:
- Create a package "wall_follower" 
- Create a python file "wall_follower_node.py" to perform the wall follower in the maze of our gopigo3 robot
- Create a launch file to initialyse all the needed nodes in our system for autonomous navigation

We create first the new package "wall_follower" with dependencies of the different message types

In [ ]:
cd ~/mpuig/pi_ws/src
catkin_create_pkg wall_follower rospy sensor_msgs geometry_msgs nav_msgs
cd ..
catkin_make

In src folder you create the python files for wall follower purposes

The instructions to perform the python program are in the notebook: https://github.com/Albert-Alvarez/ros-gopigo3/blob/lab-sessions/develop/ROS%20con%20GoPiGo3%20-%20S4.md



<img src="./Images/2_wall_follower1.png">

Create a launch folder for the launch files

In [ ]:
roslaunch wall_follower wall_follower.launch

In [ ]:
<launch>
  <node name="gopigo3" pkg="gopigo3_node" type="gopigo3_driver.py" />
  <include file="$(find ydlidar)/launch/lidar.launch" />
  <!-- Controller node -->
  <arg name="kp" default="0.5" />
  <arg name="distance_reference" default="0.3" />
  <arg name="lookahead_distance" default="0.3" />
  <arg name="forward_speed" default="0.1" />
  <arg name="theta" default="40.0" />
  <node name="wall_follower_controller" pkg="wall_follower_pkg" type="wall_follower_node.py" output="screen" >
    <param name="kp" value="$(arg kp)"/>
    <param name="distance_reference" value="$(arg distance_reference)"/>
    <param name="lookahead_distance" value="$(arg lookahead_distance)"/>
    <param name="forward_speed" value="$(arg forward_speed)"/>
    <param name="theta" value="$(arg theta)"/>
  </node>
</launch>